In [154]:
import pandas as pd
import numpy as np
def generate_custom_population(bins, labels):
    total_pop = pd.read_csv('data/municipalities/age_divided_population.csv')
    age_divided = pd.DataFrame(total_pop.groupby(['region_id', pd.cut(pop["age"], bins=bins+[110], labels=labels, include_lowest=True)]).sum('population')['population'])
    age_divided.reset_index(inplace=True)
    age_divided = age_divided.pivot(index='region_id', columns=['age'])['population']
    region_names_id = pd.read_csv("data/municipalities/Kommunenummer_navn_2020.csv", delimiter=",").drop_duplicates()
    df = pd.merge(region_names_id, age_divided, on="region_id", how='right', sort=True)
    df['population'] = df.loc[:,df.columns[2:2+len(labels)]].sum(axis=1)
    return df

def generate_labels_from_bins(bins):
    labels = []
    for i in range(len(bins)-1):
        if i == 0:
            labels.append(str(bins[i])+"-"+str(bins[i+1]))
        else:
            labels.append(str(bins[i]+1)+"-"+str(bins[i+1]))
    labels.append(str(bins[-1]+1)+"+")
    return labels

bins = [0,5,15,19,66]
df = generate_custom_population(bins, generate_labels_from_bins(bins))
df

,region_id,region,0-5,6-15,16-19,20-66,67+,population
0,301,OSLO,48242,71631,25672,473486,77749,696780
1,1101,EIGERSUND,957,1894,739,8756,2439,14785
2,1103,STAVANGER,10034,17919,6740,90971,18451,144115
3,1106,HAUGESUND,2252,4380,1831,23142,5707,37312
4,1108,SANDNES,6462,11237,4057,49483,9206,80445
...,...,...,...,...,...,...,...,...
351,5440,BERLEVÅG,43,65,42,564,214,928
352,5441,TANA,142,256,115,1715,600,2828
353,5442,NESSEBY,26,93,31,501,229,880
354,5443,BÅTSFJORD,133,214,97,1410,346,2200


In [160]:
available_vaccines = 1000
pop_weight = df[df.columns[2:-1]].to_numpy()/df.population.sum()
decision = available_vaccines * pop_weight
decision

array([[8.95011390e+00, 1.32893663e+01, 4.76280677e+00, 8.78436556e+01,
        1.44244104e+01],
       [1.77547759e-01, 3.51385012e-01, 1.37103233e-01, 1.62445996e+00,
        4.52496327e-01],
       [1.86156136e+00, 3.32442873e+00, 1.25044085e+00, 1.68774266e+01,
        3.42312822e+00],
       ...,
       [4.82365908e-03, 1.72538575e-02, 5.75128583e-03, 9.29482000e-02,
        4.24853050e-02],
       [2.46748715e-02, 3.97024248e-02, 1.79959589e-02, 2.61590743e-01,
        6.41917709e-02],
       [1.00925790e-01, 1.97027921e-01, 9.18350479e-02, 1.17585966e+00,
        3.08343131e-01]])

In [80]:
c = np.array([[[0.005, 0.02, 0.01, 0.02, 0.08, 0.04, 0.01],
    [0.02,  0.05, 0.04, 0.03, 0.2,  0.1,  0.05],
    [0.01,  0.04, 0.04, 0.02, 0.1,  0.1,  0.05],
    [0.02,  0.03, 0.02, 0.02, 0.1,  0.09, 0.03],
    [0.08,  0.2,  0.1,  0.1,  0.5,  0.4,  0.2 ],
    [0.04,  0.1,  0.1,  0.09, 0.4,  0.3,  0.1 ],
    [0.01,  0.05, 0.05, 0.03, 0.2,  0.1,  0.1 ]],
    [[0.4,   0,    0,     0,   0,    0,    0],
    [0,     0.9,  0.09,  0,   0,    0,    0],
    [0,     0.09, 0.5,   0,   0,    0,    0],
    [0,     0,    0,     1,   0,    0,    0],
    [0,     0,    0,     0,   0,    0,    0],
    [0,     0,    0,     0,   0,    0,    0],
    [0,     0,    0,     0,   0,    0,    0]],
    [[0,     0,    0,     0,   0,    0,    0],
    [0,     0,    0,     0,   0,    0,    0],
    [0,     0,    0.1,   0.2, 0.2,  0.06, 0],
    [0,     0,    0.2,   0.4, 0.4,  0.1,  0],
    [0,     0,    0.2,   0.4, 0.5,  0.1,  0],
    [0,     0,    0.06,  0.1, 0.1,  0.03, 0],
    [0,     0,    0,     0,   0,    0,    0]],
    [[0.06,  0.08, 0.1,   0.1,  0.1,  0.1,  0.03],
    [0.08,  0.1,  0.1,   0.1,  0.2,  0.1,  0.04],
    [0.1,   0.1,  0.2,   0.2,  0.2,  0.2,  0.05],
    [0.1,   0.1,  0.2,   0.2,  0.2,  0.2,  0.05],
    [0.1,   0.2,  0.2,   0.2,  0.3,  0.2,  0.07],
    [0.1,   0.1,  0.2,   0.2,  0.2,  0.2,  0.05],
    [0.03,  0.04, 0.05,  0.05, 0.07, 0.05, 0.01]]])

In [81]:
temp = np.zeros(shape=(4,7,7))
for i, mat in enumerate(c):
    for j, row in enumerate(mat):
        for k, col in enumerate(row):
            if k == 2 or k == 4:
                temp[i][j][k] = (c[i][j][k] + c[i][j][k+1]) / 2
            elif j == 2 or j == 4:
                temp[i][j][k] = (c[i][j][k] + c[i][j+1][k]) / 2
            else:
                temp[i][j][k] = col
final = []
for mat in temp:
    t_mat = np.delete(mat, 5, 0)
    t_mat = np.delete(t_mat, 5, 1)
    t_mat = np.delete(t_mat, 3, 0)
    t_mat = np.delete(t_mat, 3, 1)
    final.append(t_mat)

In [247]:
import pandas as pd
import numpy as np
from covid import utils
paths = utils.create_named_tuple('filepaths.txt')

In [242]:
def generate_contact_matrices(bins, labels):
    df = pd.read_csv('data/contact_data.csv')
    df.contact_age_0 = pd.cut(df['contact_age_0'], bins=bins+[110], labels=labels, include_lowest=True)
    df.contact_age_1 = pd.cut(df['contact_age_1'], bins=bins+[110], labels=labels, include_lowest=True)
    df_mat = pd.DataFrame(df[df.columns[:-2]].groupby(['contact_age_0', 'contact_age_1']).sum()).reset_index()
    home = pd.pivot_table(df_mat, values='home', index='contact_age_0', columns='contact_age_1').to_numpy()
    work = pd.pivot_table(df_mat, values='work', index='contact_age_0', columns='contact_age_1').to_numpy()
    school = pd.pivot_table(df_mat, values='school', index='contact_age_0', columns='contact_age_1').to_numpy()
    transport = pd.pivot_table(df_mat, values='transport', index='contact_age_0', columns='contact_age_1').to_numpy()
    leisure = pd.pivot_table(df_mat, values='leisure', index='contact_age_0', columns='contact_age_1').to_numpy()
    return home, work, school, transport, leisure

In [253]:
age_bins = [0,15,20,40,66]
age_labels = utils.generate_labels_from_bins(age_bins)

In [478]:
df = pd.read_csv('data/contact_data.csv')
df.contact_age_0 = pd.cut(df['contact_age_0'], bins=bins+[110], labels=labels, include_lowest=True)
df.contact_age_1 = pd.cut(df['contact_age_1'], bins=bins+[110], labels=labels, include_lowest=True)
df_mat = pd.DataFrame(df[df.columns[:-2]].groupby(['contact_age_0', 'contact_age_1']).sum()).reset_index()
age_pop = (df.contact_age_0.value_counts()[age_labels] + df.contact_age_1.value_counts()[age_labels])/2
matrices = []
for col in ['home', 'work', 'school', 'transport', 'leisure']:
    matrix = pd.pivot_table(df_mat, values=col, index='contact_age_0', columns='contact_age_1')
    matrix += matrix.T
    matrix.values[[np.arange(matrix.shape[0])]*2] = matrix.values[[np.arange(matrix.shape[0])]*2]/2
    matrix = np.array([[np.round(matrix[i][j]/(age_pop[i]+age_pop[j]),2) for i in age_labels] for j in age_labels])
    matrices.append(matrix)

In [481]:
matrices

[array([[0.05, 0.02, 0.09, 0.07, 0.02],
        [0.02, 0.02, 0.02, 0.04, 0.01],
        [0.09, 0.02, 0.04, 0.06, 0.02],
        [0.07, 0.04, 0.06, 0.05, 0.03],
        [0.02, 0.01, 0.02, 0.03, 0.04]]),
 array([[0.  , 0.  , 0.01, 0.01, 0.  ],
        [0.  , 0.01, 0.02, 0.01, 0.  ],
        [0.01, 0.02, 0.08, 0.12, 0.01],
        [0.01, 0.01, 0.12, 0.08, 0.01],
        [0.  , 0.  , 0.01, 0.01, 0.  ]]),
 array([[0.19, 0.02, 0.03, 0.03, 0.  ],
        [0.02, 0.14, 0.01, 0.02, 0.  ],
        [0.03, 0.01, 0.01, 0.01, 0.  ],
        [0.03, 0.02, 0.01, 0.01, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]]),
 array([[0.02, 0.01, 0.01, 0.01, 0.  ],
        [0.01, 0.02, 0.01, 0.01, 0.  ],
        [0.01, 0.01, 0.01, 0.01, 0.  ],
        [0.01, 0.01, 0.01, 0.01, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]]),
 array([[0.08, 0.03, 0.06, 0.05, 0.02],
        [0.03, 0.08, 0.05, 0.02, 0.01],
        [0.06, 0.05, 0.08, 0.11, 0.02],
        [0.05, 0.02, 0.11, 0.08, 0.06],
        [0.02, 0.01, 0.02, 0.06,

In [22]:
df.to_csv('data/fhi_data_daily.csv', index=False)

In [10]:
import pandas as pd
df = pd.read_csv('data/fhi_data_daily.csv')
df

,date,year,week,r0_average,r0_conf_95_low,r0_conf_95_high,H_cumulative,H_new,ICU_cummulative,ICU_new,...,vaccine_supply_new,alpha_s,alpha_e1,alpha_e2,alpha_a,alpha_i,w_c1,w_c2,w_c3,w_c4
0,2/21/2020,2020,8,3.10,2.40,3.90,0,0,0,0,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29
1,2/22/2020,2020,8,3.10,2.40,3.90,0,0,0,0,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29
2,2/23/2020,2020,9,3.10,2.40,3.90,0,0,0,0,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29
3,2/24/2020,2020,9,3.10,2.40,3.90,0,0,0,0,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29
4,2/25/2020,2020,9,3.10,2.40,3.90,0,0,0,0,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,4/7/2021,2021,15,0.86,0.76,0.94,3748,29,717,4,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29
412,4/8/2021,2021,15,0.86,0.76,0.94,3776,28,721,4,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29
413,4/9/2021,2021,15,0.86,0.76,0.94,3799,23,723,2,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29
414,4/10/2021,2021,15,0.86,0.76,0.94,3805,6,725,2,...,0,1,1,1,1,0.1,0.31,0.24,0.16,0.29


In [11]:
df.w_c1.unique()

array([0.31, 0.5 , 2.  ])

In [12]:
mapping = {0.31: 0.5, 0.5: 0.1, 2.0: 1.0}
df.w_c1 = df.w_c1.map(mapping)